In [2]:
# imports
!pip install xgboost
import pandas as pd
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import make_scorer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from scipy import stats

  Using cached https://files.pythonhosted.org/packages/4f/4c/4969b10939c4557ae46e5569d07c0c7ce772b3d6b9c1401a6ed07059fdee/xgboost-0.81.tar.gz
  Running setup.py bdist_wheel for xgboost ... done
  Stored in directory: /Users/saranade/Library/Caches/pip/wheels/8a/89/0c/4870bb69132698f40889fa956f92e630a36c1fd7173fcc759f
Successfully built xgboost
distributed 1.21.8 requires msgpack, which is not installed.
grin 1.2.1 requires argparse>=1.1, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
all_data = pd.read_csv('train.csv')

trainables = all_data.filter(['OverallQual','GrLivArea','GarageCars','TotalBsmtSF','1stFlrSF','FullBath','YearBuilt'])

In [4]:
#missing data
total = trainables.isnull().sum().sort_values(ascending=False)
percent = (trainables.isnull().sum()/trainables.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
YearBuilt,0,0.0
FullBath,0,0.0
1stFlrSF,0,0.0
TotalBsmtSF,0,0.0
GarageCars,0,0.0
GrLivArea,0,0.0
OverallQual,0,0.0


In [7]:
# model it

# fix skewedness in Total Bsmt SF
# all_data['HasBsmt'] = pd.Series(len(all_data['TotalBsmtSF']), index=all_data.index)
# all_data['HasBsmt'] = 0 
# all_data.loc[all_data['TotalBsmtSF']>0,'HasBsmt'] = 1
# all_data.loc[all_data['HasBsmt']==1,'TotalBsmtSF'] = np.log(all_data['TotalBsmtSF'])
    
    
# Log of GrLivArea, 1stFlrSF, YearBuilt
all_data['GrLivArea'] = np.log(all_data['GrLivArea'])
all_data['1stFlrSF'] = np.log(all_data['1stFlrSF'])
all_data['YearBuilt'] = np.log(all_data['YearBuilt'])

# collect data
trainables = all_data.filter(['OverallQual','GrLivArea','GarageCars','TotalBsmtSF','1stFlrSF','FullBath','YearBuilt'])
trainables = np.matrix(trainables)
salePrice = np.array(all_data['SalePrice'])

# No 0s, ints between 1-10
# res = stats.probplot(all_data['OverallQual'], plot=plt)

# No 0s, should log
# res = stats.probplot(all_data['GrLivArea'], plot=plt)

# Some 0s, no log
# res = stats.probplot(all_data['GarageCars'], plot=plt)

# Some 0s, need log
# res = stats.probplot(all_data['TotalBsmtSF'], plot=plt)

# No 0s, should log
# res = stats.probplot(all_data['1stFlrSF'], plot=plt)

# Some 0s, no log
# res = stats.probplot(all_data['FullBath'], plot=plt)

# No 0s, log
# res = stats.probplot(all_data['YearBuilt'], plot=plt)

n_folds = 5
scorer = make_scorer(mean_squared_error,greater_is_better = False)

x_train,x_test,y_train,y_test = train_test_split(trainables,salePrice,test_size = 0.3,random_state= 0)

lr = LinearRegression()
lr.fit(x_train,y_train)
predictions = lr.predict(x_test)

gboost = XGBRegressor();
gboost.fit(x_train, y_train, verbose=False)

cross_val_boost = cross_val_score(gboost,x_test,y_test)
rmse_boost = np.sqrt(cross_val_boost.mean())

cross_val = cross_val_score(lr,x_test,y_test)
rmse = np.sqrt(cross_val.mean())
print(rmse)
print(rmse_boost)

0.8235291237816716
0.8380843685665583
